In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, generate_data_commitment

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')
data_commitment_path = os.path.join('shared/data_commitment.json')
precal_witness_path = os.path.join('shared/precal_witness_arr.json')

=======================  ZK-STATS FLOW =======================

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')
create_dummy(data_path, dummy_data_path)

In [5]:
# use large scale here to get the output really close to the value in dataset, which is important to Mode.
# large scale doesn't blowup circuit size in Mode, so fine.
scales = [8]
selected_columns = ['col_name']
generate_data_commitment(data_path, scales, data_commitment_path)

In [6]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State

def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x = data[0]
    return s.mode(x)
# IMPORTANT: Here, we use traditional Mode, so there is no result_error at all, since Mode pick the result from dataset, instead of calculating the result
# But we still have error stuffs here to just conform to computation_to_model function, though the error's not used at all.
error = 0

# Prover/ data owner side
_, prover_model = computation_to_model(computation, precal_witness_path, True, selected_columns, error)
prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model, prover_model_path, scales, "resources", settings_path)



/Users/jernkun/Desktop/zk-stats-lib/zkstats/ops.py:228: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  result = torch.tensor(mode_within(x_1d, 0))
/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:254: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Desktop/zk-stats-lib/zkstats/ops.py:246: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  for ele i

==== Generate & Calibrate Setting ====




 <------------- Numerical Fidelity Report (input_scale: 8, param_scale: 8, scale_input_multiplier: 10) ------------->

+----------------+----------------+-----------+----------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error     | median_error   | max_error | min_error      | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+----------------+----------------+-----------+----------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| -0.00039100647 | -0.00078201294 | 0         | -0.00078201294 | 0.00039100647  | 0.00078201294    | 0.00078201294 | 0             | 0.00000030577212   | -0.000008727823    | 0.000008727823         |
+----------------+----------------+-----------+----------------+---------------

scale:  [8]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":8,"param_scale":8,"scale_rebase_multiplier":10,"lookup_range":[-50326,512],"logrows":16,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Fixed","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":12309,"total_assignments":24619,"total_const_size":8263,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,8],"model_input_scales":[8],"module_sizes":{"kzg":[],"poseidon":[3936,[1]]},"required_lookups":[{"Cast":{"scale":256.0}},"ReLU",{"GreaterThan":{"a":0.0}}],"required_range_checks":[],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1715324275836}


In [7]:
_, verifier_model = computation_to_model(computation, precal_witness_path, False, selected_columns, error)
verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path,verifier_model, verifier_model_path)

In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 6.974064111709595 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 44.80078125
==== Generating Proof ====
proof:  {'instances': [['92c6e4646b9211cd10c19d44838585e3eaeb2d63822ebad95594ff556580b100', '0100000000000000000000000000000000000000000000000000000000000000', 'cd2c000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x057d4411b88724a9f8f8c021c09f2f65d9b3281876bdb8805c54de990f7c27c42be6c32899336ded0db1486428fef62a199092e2083ba84a421f51cdc94ffe1205bca33b34e4bb82830089d88cd40f14d334860a0956d3535ac2acbcad6980181cb1f15d87a5f5a2dc9fa5e822a63153f150ddfee51c4c42546c140767689d340ca80f1efe3a013d1836c094397f9bc6a3d689dbb18021a71d31ccfce3bb63f11ccdd5eb822d5137ac61f97f71bf76c63b618e4162b91dcef4a4751b19bbb76a0756734656b95e2a436bf1a403384f8558cee21667ba01396f20cbc2e18e64622980c1054387aca0e590a7a746ca302acb634271897b8b7a637e67b98532453b10d12879a43fed3339b52bb0acadeb0ceadb7e6fcedd67d89e31e1f366835d801c261445e

In [9]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, data_commitment_path)
print("Verifier gets result:", res)

Verifier gets result: [44.80078125]
